In [15]:
# this file will check which neural network works the best for our use case
# For this purpose I will just run the neural network with a predefined parameters and inside a loop
# let's just keep the parameters to zero

# also jsut shuffle the data and then use last 20% of the data for testing

In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('/Users/carbon/Desktop/pytorch NN/Visualization/data.csv')

In [18]:
# shuffle the data inside dataframe
df = df.sample(frac=1).reset_index(drop=True)

In [19]:
# create a training and testing set
train = df.iloc[:int(len(df)*0.5)]
test = df.iloc[int(len(df)*0.5):]

## The Neural Network

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import json

In [21]:
# funtion that takes parameters, creates a zeroed NN, train and return everything

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(9,16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 5)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = nn.functional.leaky_relu(self.fc3(x))
        x = nn.functional.softmax(self.fc4(x), dim=1)
        return x

X_train = train.drop('y', axis=1).values
y_train = train['y'].values
X_test = test.drop('y', axis=1).values
y_test = test['y'].values

# standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorch data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, momentum=0.1)

# turn all model parameters to zero
for param in model.parameters():
    param.data = torch.zeros_like(param.data)

# Train the PyTorch model
for epoch in range(10):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Test the PyTorch model
model.eval()
with torch.no_grad():
    test_outputs = model(torch.tensor(X_test, dtype=torch.float32))
    _, predicted = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test, predicted.numpy())
    print('Test accuracy:', test_accuracy)


Test accuracy: 0.4620131794567832
